In [2]:
import numpy as np
import pandas as pd
import scipy.linalg
import os
import time
import sklearn.covariance
import pickle
from collections import defaultdict

import Util  # Util module from this library

### Load data

In [ ]:
util_loaded_data = Util.load_data()
ret, ret_nonan, univ, tradeidx, dates = util_loaded_data

### Saving data for past returns

Given T (lookback period) and N (number of stocks), save past returns over relevant period, indexed by h. These returns are used for prediction.

In [ ]:
# If the paths don't exist, then create them.
# This is where the data is saved.

assert os.path.exists('./pastRets')
assert os.path.exists('./outRets')

In [ ]:
def save_past_information(T, N, util_loaded_data, save_KT=True):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRets = []
    KT_covs = []
    
    for h in range(len(univ)):
        print(h, end = ' ')
        pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
        pastRetFileName = "pastRets/{}_{}_{}_pastRet.pkl".format(T, N, h)
        with open(pastRetFileName, 'wb') as f:
            pickle.dump(pastRet, f)
        pastRets.append(pastRet)
        
        if save_KT:
            KT_cov = Util.kendall_cov(pastRet)
            KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
            with open(KT_cov_file_name, 'wb') as f:
                pickle.dump(KT_cov, f)
            KT_covs.append(KT_cov)
    
    with open("pastRets/{}_{}_pastRets.pkl".format(T,N), 'wb') as f:
        pickle.dump(pastRets, f)

    if save_KT:
        with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)

In [ ]:
for T in [25, 50, 100, 200, 500, 1260]:
    for N in [25, 50, 100, 200, 500, 1000]:
        save_past_information(T, N, util_loaded_data)

In [ ]:
# Saves the out of sample returns for evaluation time.
def save_outrets(N, util_loaded_data, P=1):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    all_outrets = []
    for h in range(360):
        outret = get_invest_period(h, P, N, univ, tradeidx, ret)
        all_outrets.append(outret)
    with open('outRets/{}_outRets.pkl'.format(N), 'wb') as f:
        pickle.dump(all_outrets, f)

for N in [25, 50, 100, 200, 500, 1000]:
    save_outrets(N, util_loaded_data)

### Using estimators to generate portfolios

An estimator is a function that takes in the past returns (a N by T matrix of past returns) and returns an estimated covariance (N by N) of the stocks. 2 methods take in a Kendall-Tau covariance matrix of the past returns instead of the raw past returns: MTP2_cov_wrapper and CLIME_cov_wrapper.

We demonstrate below how to compute the portfolio for a given estimator and choice of N, T over the out of sample time period (indexed by h).

In [ ]:
from estimators import (MTP2_wrapper, MTP2_cov_wrapper,
                        LRPS_wrapper,
                        CLIME_wrapper, CLIME_cov_wrapper,
                        LS_wrapper, NLS_wrapper,
                        POET_wrapper, POET_5_wrapper,
                        glasso_wrapper,
                        get_AFM_estimator)

In [ ]:
# A dictionary mapping estimator name to the estimator function.
all_estimators = {
    'MTP2_KT': MTP2_cov_wrapper,
    'CLIME_KT': CLIME_cov_wrapper,
    'glasso': glasso_wrapper,
    'MTP2': MTP2_wrapper,
    'CLIME': CLIME_wrapper,
    'LS': LS_wrapper,
    'NLS':  NLS_wrapper,
    'LRPS': LRPS_wrapper,
    'AFM_NL': get_AFM_estimator(5, 'NLS', tradeidx),
    'AFM_LS': get_AFM_estimator(5, 'LS', tradeidx),
    'POET': POET_wrapper,
    'POET_5': POET_5_wrapper
}

In [ ]:
T = 25
N = 200
estimator_name = 'glasso'

# First load the past returns.
with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)
    assert len(pastRets) == 360

with open('outRets/{}_outRets.pkl'.format(N), 'rb') as f:
    outRets = pickle.load(f)

estimator = all_estimators[estimator_name]
estimator_portfolio_return = []
for h in range(360):
    # Get estimated covariance.
    cov = estimator(pastRets[h])
    # Get optimal portfolio weights.
    w = optimal_weights(cov)
    # Calculate the return of the portfolio with given weights.
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)

# prints normalized (by percent per year) mean and std and information ratio
# ratio of return of  portfolio
print(Util.get_IR(estimator_portfolio_return))

In [ ]:
# If using momentum signal for estimating portfolio weights
# (instead of only covariance), use the following code.

estimator = all_estimators[estimator_name]
estimator_portfolio_return = []
for h in range(360):
    # Gets momentum signal of stocks.
    m = get_momentum_signal(h, N, univ, tradeidx, ret)
    b = np.mean(m)
    # Get estimated covariance.
    cov = estimator(pastRets[h])
    # Get optimal portfolio weights.
    w = optimal_weights_momentum(m, cov, b)
    # Calculate the return of the portfolio
    curret = Util.retConstShare(outRets[h], w)
    estimator_portfolio_return.append(curret)